# Timeseries with recurrent neural networks

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore", UserWarning)
%matplotlib inline
plt.style.use('dark_background')

In [ ]:
import mltrainer
mltrainer.__version__

# 1.0 Sunspots
## 1.1 Loading data

In [ ]:
from mads_datasets import DatasetFactoryProvider, DatasetType
sunspotfactory = DatasetFactoryProvider.create_factory(DatasetType.SUNSPOTS)
sunspotfactory.download_data()

First let's look at the raw data.

In [ ]:
import numpy as np
f = sunspotfactory.filepath
f.exists(), f

In [ ]:
spots = np.genfromtxt(f, usecols=(2,3))
spots

This is data on sunspots, since 1749, in a pandas dataframe. Let us cast this to a `float32` Torch Tensor.

We first need to get the numpy data out of the pandas, and the cast that to a float32 Tensor.

In [ ]:
import torch
tensordata = torch.from_numpy(spots[:, 1]).type(torch.float32)
tensordata

In [ ]:
plt.plot(tensordata);

## 1.2 Train-test split
There seems to be some pattern. But also a lot of noise. We need to take this into account for our final model: it might be hard to get very high accuracy, because we dont have any additional features (could you think of possible relevant features?).

So let's make a train-test split, and normalize on the trainset. Note that we split on past-future! (Why is this important?)

In [ ]:
# Train test split
split = 2500
train = tensordata[:split]

norm = max(train)
test = tensordata[split:]

train = train / norm
test = test/ norm

What would go wrong if you normalize on the max of the testset?

In [ ]:
# reduce steps back to months, starting in 1749
idx0 = torch.arange(0, len(train)) / 12 + 1749
idx1 = torch.arange(0, len(test)) / 12 + idx0.max()
fig, axs = plt.subplots(2, sharex=True)
axs[0].plot(idx0, train)
axs[1].plot(idx1,test)

## 1.3 Windowing

Now, let's use our windowed datagenerator.

Regarding the windowsize, we will have to ask ourselves: what is a reasonable time, we will need to look into the past to be able to predict the future? If we make it too long, we will get irrelevant data (eg, does the amount of sunspots a hundred years still impact significantly the sunspots in 2020? If you think so, you should increase your window by a lot!)

In [ ]:
from mads_datasets import datatools

window_size = sunspotfactory.settings.window_size
print(f"windowsize: {window_size}")

idx = datatools.window(train, window_size)
trainset = train[idx]
idx = datatools.window(test, window_size)
testset = test[idx]
trainset.shape, testset.shape

Ok, that is looking good. We squeezed out 2475 training examples out of this long, single timeseries.

## 1.4 A custom dataset

Let's make the data 3D, just as our timeseries model will need to have it. We just have one feature, so that is just a dimension of 1.

In [ ]:
trainset = trainset[...,None]
testset = testset[..., None]
trainset.shape, testset.shape

We can implement a Dataset in PyTorch. 
According to the [documentation](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files)
> A custom Dataset class must implement three functions: `__init__`, ` __len__`, and `__getitem__`


The functions with the double underscores are called "dunder" function. 
The `__len__` function is the function that is called when you do `len(object)`, the `__getitem__` function is called when you do `object[idx]` and `idx` is passed as an argument.

We can inherit the `Dataset` class, and if we implement a `__len__` and `__getitem__` function, we are good.

In [ ]:
from mads_datasets.base import DatasetProtocol
from typing import Tuple
Tensor = torch.Tensor

class SunspotDataset(DatasetProtocol):
    def __init__(self, data: Tensor, horizon: int) -> None:
        self.data = data
        self.size = len(data)
        self.horizon = horizon

    def __len__(self) -> int:
        return self.size

    def __getitem__(self, idx: int) -> Tuple[Tensor, Tensor]:
        # get a single item
        item = self.data[idx]
        # slice off the horizon
        x = item[:-self.horizon,:]
        y = item[-self.horizon:,:].squeeze(-1) # squeeze will remove the last dimension if possible.
        return x, y

Let us test if this works as expected

In [ ]:
datashape = (100, 10, 2)
dummydata = torch.randint(0, 10, datashape)
dummydataset = SunspotDataset(dummydata, horizon=2)
len(dummydataset) # uses the __len__ method

In [ ]:
x, y = dummydataset[0] # uses the __getitem__ method
x.shape, y.shape # no squeeze because we have dimension 2 as the last dimension

Ok, now we apply this on the trainset.

In [ ]:
trainset.shape

In [ ]:
horizon = 3
traindataset = SunspotDataset(trainset, horizon=horizon)
testdataset = SunspotDataset(testset, horizon=horizon)

In [ ]:
traindataset.data.shape

In [ ]:
x, y = traindataset[1]
x.shape, y.shape # here we see the squeeze happening

## 1.5 A Dataloader
This seems to work as well. 
Again, everything is wrapped into the dataset factory, which will give you windowed and batched streamers.

In [ ]:
sunspotfactory.settings

In [ ]:
from mltrainer.preprocessors import BasePreprocessor
preprocessor = BasePreprocessor()

streamers = sunspotfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

Can you make sense of all the output?
What is going on? Is that what you expected?

In [ ]:
x, y = next(iter(trainstreamer))
type(x), len(x), x[0].shape, type(y)

In [ ]:
x.shape, y.shape

What do you think these numbers are?

In [ ]:
len(train), len(valid)

# 2 Exploring baseline models
## 2.1 Naive models

The most basic prediction is a Naive model. 

- What is a naive prediction of the temperature for tomorrow? Well, the temperature of today.
- The stock market for tomorrow? Same as today.
- Bitcoin for march 16th of 2028? Same as march 15th 2028.

In general, this is a super simple model without any parameters but often much better than you would expect, because the temparature of today will be close to the temperature of tomorrow.

In general, you can say that f is a function that you feed the current moment, which is a real number, and it will map it to one step into the future, which is also a real number.

$$ f: x_t \in \mathcal{R} \to x_{t+1} \in \mathcal{R}$$

but $f$ is just the Identity function $f(x) = x$, so one step into the future is the same as the current timestep.

This gives rise to the MASE, the Mean Absolute Scaled Error: It is the mean absolute error of the forecast values (the actual model) 
$\frac{1}{J}\sum_j|e_j|$, but scaled by the mean absolute error of the in-sample one-step naive forecast $|Y_t-Y_{t-1}|$ (the naive model):

$$ MASE = \frac{\frac{1}{J}\sum_j|e_j|}{
\frac{1}{T-1} \sum_{t=2}^{T}|Y_t-Y_{t-1}|} $$

This takes a moment in time $t$, and calculates the difference with one step back $t-1$.

In other words, the MASE compares the MAE of your actual model to the MAE of the naive model. 

E.g. if the error of your model is 1, and the error of the naive model is 10, your MASE is 0.1, meaning you are much better than the naive prediction.

MASE values above 1 are really bad, because it means the naive function outperformed your actual model, everything below (ideally close to 0) is an improvement.

We will implement this ourselves. First the naive prediction:

In [ ]:
Tensor = torch.Tensor

def naivepredict(x: Tensor, horizon: int) -> tuple[Tensor, Tensor]:
    assert horizon > 0
    yhat = x[...,-horizon:, :].squeeze(-1)
    return yhat

In [ ]:
yhat = naivepredict(x, horizon) #this should be the same horizon you used to create the dataset
yhat.shape

With the naive prediction, we can calculate the MAE

In [ ]:
def mae(y: np.ndarray, yhat: np.ndarray) -> float:
    return np.mean(np.abs(y-yhat))

In [ ]:
mae(y.detach().numpy(), yhat.detach().numpy())

This error is not deterministic, because of the shuffling in the dataset.

Now, we want to calculate the naive MAE for every batch, to get the average, expected MAE for the naive prediction

In [ ]:
def naivenorm(train , horizon: int):
    elist = []
    streamer = train.stream()

    for _ in range(len(train)):
        x, y = next(iter(streamer))
        yhat = naivepredict(x, horizon)
        e = mae(y.numpy(), yhat.numpy())
        elist.append(e)
    return torch.mean(torch.tensor(elist))
naivenorm(train, 1)

This means, a MAE close to this number is not impressive at all. We need to get below this if we want to go beyond the naive prediction.
We can wrap everything in a class, so it is neatly organized.

This example also illustrates two other dunder methods, `__repr__` and `__call__`

In [ ]:
from mads_datasets.base import BaseDatastreamer
Tensor = torch.Tensor
class MASE:
    def __init__(self, train: BaseDatastreamer, horizon: int):
        self.scale = self.naivenorm(train, horizon)

    def __repr__(self) -> str:
        # __repr__ is what is returned if you print the class to the screen
        return f"MASE(scale={self.scale:.3f})"

    def naivenorm(self, train: BaseDatastreamer, horizon: int) -> Tensor:
        elist = []
        streamer = train.stream()
        for _ in range(len(train)):
            x, y = next(iter(streamer))
            yhat = self.naivepredict(x, horizon)
            e = self.mae(y.numpy(), yhat.numpy())
            elist.append(e)
        return torch.mean(torch.tensor(elist))

    def naivepredict(self, x: Tensor, horizon: int) -> Tuple[Tensor, Tensor]:
        assert horizon > 0
        yhat = x[...,-horizon:, :].squeeze(-1)
        return yhat

    def mae(self, y: np.ndarray, yhat: np.ndarray) -> float:
        return np.mean(np.abs(y-yhat))

    def __call__(self, y: np.ndarray, yhat: np.ndarray) -> float:
        # __call__ lets us call the class as a function
        return self.mae(y, yhat) / self.scale

In [ ]:
mase = MASE(train, horizon)
mase # __repr__ in action

In [ ]:
mase(y.numpy(), yhat.numpy()) # __call__ in action

## 2.2 Linear model
Ok, we have everything in place. Let's start with a slightly more advanced model than a naive one.

We start with a linear baselinemodel, of just a single `Linear` layer with one unit. This is equal to a linear model.

**NOTE** A Neural Network (consisting of Linear layers AND activation layers) is NOT a simple model; if your model is deep & wide enough, this is a universal function approximator! That means a neural network is capable, in theory, of approximating EVERY function. In this case we use a single layer, with a minimum amount of units; that's why this is a simple model!

Check out the sourcecode (tip: VS code let's you do so easily by pressing F12, or by right-clicking the mouse on a function and selecting `Go to Definition`)

In [ ]:
from mltrainer import rnn_models, metrics
observations = window_size - horizon
model = rnn_models.BaseModel(observations=observations, horizon=horizon)
x, y = next(iter(trainstreamer))
x.shape, y.shape

Let's have a look at our random model

In [ ]:
yhat = model(x)
yhat.shape

In [ ]:
mase = metrics.MASE(train, horizon)
mase(y.detach().numpy(), yhat.detach().numpy())

Oh boy! Untrained, this is actually much much worse than a naive model... Let's start training...

First, we add a regular MAE too.

In [ ]:
mae = metrics.MAE()
mae(yhat.detach().numpy(), y.detach().numpy())

In [ ]:
from loguru import logger
import sys
logger.remove()
logger.add(sys.stderr, level="SUCCESS")

In [ ]:
from mltrainer import TrainerSettings, ReportTypes
from pathlib import Path

log_dir = Path("logs/rnn/").resolve()
settings = TrainerSettings(
    epochs=100,
    metrics=[mase, mae],
    logdir=log_dir,
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
    scheduler_kwargs = {"factor": 0.5, "patience": 5},
    earlystop_kwargs = {
        "save": False,
        "verbose": True,
        "patience": 10,
    }
)

In [ ]:
device = "cpu" # faster than mps in this case

In [ ]:
from mltrainer import Trainer

observations = window_size - horizon
model = rnn_models.BaseModel(
    observations=observations,
    horizon=horizon,
)

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=torch.nn.MSELoss(),
    optimizer=torch.optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau,
    device=device,
    )
trainer.loop()

If you have a look at Tensorboard, you can see the model is learning!

In [ ]:
x, y = next(iter(validstreamer))
yhat = model(x)
mae(y.detach().numpy(), yhat.detach().numpy()), mase(y.detach().numpy(), yhat.detach().numpy())

And, yes, the MASE dropped below 1! This is a good start.

Let's visualize this.

In [ ]:
k = 4
fig, ax = plt.subplots(2, 2, figsize=(10, 10))
axs = ax.ravel()

timesteps = x.shape[1]
idx0 = [*range(timesteps)]
idx1 = [(i + timesteps) for i in range(horizon)]
for i in range(k):
    axs[i].plot(idx0, x[i])
    axs[i].plot(idx1, yhat[i].detach(), color="red", label="predict")
    axs[i].plot(idx1, y[i], color="green", label="actual")
handles, labels = axs[i].get_legend_handles_labels()
fig.legend(handles, labels)

Well, it is not perfect, obviously. But it does seem to capture the trend more often that random.

## 2.3 Simple RNN

Let's invoke the basic RNN without gates.

Input size is 1, because we have 1 feature. We follow [the batch_first convention](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html). Hidden size is the inner dimension of the RNN, and thus the dimensionality of the output.

What would happen if we feed this data with dimensions `(batch, sequence_length)`?

In [ ]:
x = torch.randn((32, 25))
rnn = torch.nn.RNN(input_size=1, hidden_size=1, batch_first=True)
try:
    yhat = rnn(x)
except RuntimeError as e:
    print(e)

Note how it expects 3 dimensions : `(batch, sequence_length, features)`, even though we have just one feature.

In [ ]:
x, y = next(iter(trainstreamer))
x.shape, y.shape

In [ ]:
out, hidden = rnn(x)
out.shape, hidden.shape

This works as expected. Let's increase the hidden dimension (why would you do that?)

In [ ]:
rnn = torch.nn.RNN(input_size=1, hidden_size=10, batch_first=True)
out, hidden = rnn(x)
out.shape, hidden.shape

And let's add multiple stacked layers of RNN (can you visualize how this would work?)

In [ ]:
rnn = torch.nn.RNN(
    input_size=1,
    hidden_size=10,
    num_layers=3,
    batch_first=True)
out, hidden = rnn(x)
out.shape, hidden.shape

To get the output of the last timestep, we need to throw away the rest.

In [ ]:
out = out[:,-1,:]
out.shape

So, let's wrap this up in a model. Study the implementation in `rnn_models.BaseRNN`.

Do you understand what the linear layer does? What would happen if you remove it?

In [ ]:
from mltrainer import metrics, rnn_models

mase = metrics.MASE(train, horizon)
mae = metrics.MAE()

In [ ]:
settings.epochs = 10
device = "cpu" # still faster than mps

In [ ]:
observations = window_size - horizon
model = rnn_models.BaseRNN(
    input_size=1,
    hidden_size=10,
    num_layers=3,
    horizon=horizon,
    )

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=torch.nn.MSELoss(),
    optimizer=torch.optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau,
    device=device,
    )
trainer.loop()

Does this improve over the linear model?
By how much? Is there a difference between loss and MASE? 

Use the tensorboard to find this out!

What does it mean?

Can you improve the model by tweaking it?